## basics (before midsem portions)

##### theta omega(diff policy using states update)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gymnasium as gym

In [2]:
env = gym.make("CartPole-v1")
env.reset()

(array([ 0.00432969,  0.04707438, -0.03036953, -0.03217735], dtype=float32),
 {})

In [3]:
episode_reward = []
for i in range(10):
    state,info = env.reset()
    total_reward =0
    done = False
    while not done:
        theta = state[2]
        action= 0 if theta <0 else 1
        state,reward,done,_,_ = env.step(action)
        total_reward += reward
    episode_reward.append(total_reward)


In [4]:
episode_reward

[38.0, 38.0, 38.0, 58.0, 42.0, 39.0, 40.0, 38.0, 47.0, 46.0]

In [5]:
episode_reward = []
for i in range(10):
    state,info = env.reset()
    total_reward =0
    done = False
    while not done:
        omega = state[3]
        action= 0 if omega <0 else 1
        state,reward,done,_,_ = env.step(action)
        total_reward += reward
    episode_reward.append(total_reward)

In [6]:
episode_reward

[141.0, 201.0, 219.0, 211.0, 283.0, 189.0, 256.0, 135.0, 172.0, 157.0]

In [ ]:
#if mountain car then with position and velocity and poistion is state[0] and velocity is state[1]

### DFS and BFS

In [14]:
from collections import deque
def bfs(start,goal,graph):
    queue = deque([(start,[start])])#node,path
    explored = set()
    explored.add(start)
    
    while queue:
        node,path = queue.popleft()
        if node == goal:
            return path
        for neighbour in graph[node]:
            if neighbour not in explored:
                queue.append((neighbour,path+[neighbour]))
                explored.add(neighbour)
    
    return []

In [15]:
graph = {
    'A': ['B', 'C'],
    'B': ['D', 'E'],
    'C': ['F'],
    'D': [],
    'E': ['F'],
    'F': []
}
print(bfs('A', 'F', graph))

['A', 'C', 'F']


In [16]:
def dfs(start,goal,graph):
    stack= [(start,[start])]
    explored = set()
    explored.add(start)
    while stack:
        node,path = stack.pop()
        if node == goal:
            return path
        for neighbour in graph[node]:
            if neighbour not in explored:
                stack.append((neighbour,path+[neighbour]))
                explored.add(neighbour)
    return []
print(dfs('A', 'F', graph))

['A', 'C', 'F']


### Iterative Deepening DLS and minmax

In [20]:
def iterative_deepening(graph, node, target, path=None, limit=5):
    if path is None:
        path = []

    path = path + [node]  # ⬅️ create a fresh copy instead of modifying in-place

    if node == target:
        return True, path

    if limit <= 0:
        return False, None

    for neighbour in graph[node]:
        if neighbour not in path:
            found, result_path = iterative_deepening(graph, neighbour, target, path, limit - 1)
            if found:
                return True, result_path

    return False, None  # No need to pop due to path copy


def DLS(start,target,graph):
    depth = 1
    while True:
        found,path = iterative_deepening(graph,start,target,None,depth)
        if found:
            return path
        depth+=1

In [21]:
graph = {
    'A': ['B', 'C'],
    'B': ['D', 'E'],
    'C': ['F'],
    'D': [],
    'E': ['F'],
    'F': []
}

start_node = 'A'
goal_node = 'F'

result_path = DLS(start_node, goal_node, graph)
print("Path to goal:", result_path)

Path to goal: ['A', 'C', 'F']


In [22]:
class Tree:
    def __init__(self,key):
        self.key = key
        self.children = []
    def insert(self,node):
        self.children.append(node)
        
# Tree creation with proper node references
root = Tree(6)

node1 = Tree(3)
node2 = Tree(6)
node3 = Tree(5)

root.insert(node1)
root.insert(node2)
root.insert(node3)

node1.insert(Tree(5))
node1.insert(Tree(3))

node2.insert(Tree(6))
node2.insert(Tree(7))

node3.insert(Tree(5))
node3.insert(Tree(8))

# Function to print the tree (for verification)
def print_tree(node, level=0):
    print("  " * level + str(node.key))  # Indentation for hierarchy
    for child in node.children:
        print_tree(child, level + 1)

# Print the tree structure
print_tree(root)

6
  3
    5
    3
  6
    6
    7
  5
    5
    8


In [24]:
def minmax(node,maximizing,alpha,beta):
    if not node.children:
        return node.key
    if maximizing:
        max_val = float('-inf')
        for children in node.children:
            val = minmax(children,False,alpha,beta)
            max_val = max(max_val,val)
            alpha = max(alpha,max_val)
            if alpha >= beta:
                break
        return max_val
    else:
        min_val = float('inf')
        for children in node.children:
            val = minmax(children,True,alpha,beta)
            min_val = min(min_val,val)
            beta = min(beta,min_val)
            if alpha >= beta:
                break
        return min_val

def run_min_max(root):
    val = minmax(root,True,float('-inf'),float('inf'))
    return val

# Running Alpha-Beta Minimax
ans = run_min_max(root)
print("Minimax Value with Alpha-Beta Pruning:", ans)

Minimax Value with Alpha-Beta Pruning: 6


In [ ]:
#DLS on env
def DLS(state, env, target_state, depth, visited):
    if state == target_state:
        return True, [state]
    if depth <= 0:
        return False, None

    for action in env.action_space:
        next_state = transition(state, action,env)
        if next_state not in visited:
            found, path = DLS(next_state, env, target_state, depth - 1, visited | {next_state})
            if found:
                return True, [state] + path

    return False, None
def transition(state, action, env):
    outcomes = env.P[state][action]  # list of (prob, next_state, reward, done)
    best = max(outcomes, key=lambda x: x[0])  # pick highest-probability one
    return best[1]  # return the most likely next_state
